In [41]:
from AppalachianRL.envs.trail import AppalachianTrailEnv


env = AppalachianTrailEnv()

obs, info = env.reset()
done = False

while not done:
    action = env.action_space.sample()
    obs, reward, done, trunc, info = env.step(action)
    env.render()


Day 1: 2200.0 miles left | Energy 100.0 | Food 10.0 | Weather 0
Day 2: 2197.0 miles left | Energy 76.0 | Food 9.5 | Weather 1
Day 3: 2197.0 miles left | Energy 100.0 | Food 8.5 | Weather 2
Day 4: 2196.0 miles left | Energy 90.0 | Food 8.3 | Weather 0
Day 5: 2191.0 miles left | Energy 55.0 | Food 7.5 | Weather 0
Day 6: 2191.0 miles left | Energy 75.0 | Food 6.5 | Weather 0
Day 7: 2191.0 miles left | Energy 95.0 | Food 5.5 | Weather 0
Day 8: 2191.0 miles left | Energy 100.0 | Food 5.5 | Weather 0
Day 9: 2190.0 miles left | Energy 85.0 | Food 5.3 | Weather 2
Day 10: 2185.0 miles left | Energy 50.0 | Food 4.5 | Weather 0
Day 11: 2180.0 miles left | Energy 8.0 | Food 3.7 | Weather 1
Day 12: 2180.0 miles left | Energy 32.0 | Food 2.7 | Weather 1
Day 13: 2180.0 miles left | Energy 44.0 | Food 2.7 | Weather 1
Day 14: 2179.0 miles left | Energy 32.0 | Food 2.5 | Weather 1
Day 15: 2174.0 miles left | Energy 0.0 | Food 1.7 | Weather 0
